In [2]:
# usual imports 
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import scipy.stats as sts
import pandas as pd
for gpu in tf.config.experimental.list_physical_devices('GPU'):
      tf.config.experimental.set_memory_growth(gpu, True)
import sys
import pickle
from matplotlib import pyplot as plt

# link to the packages
sys.path.append('./../packages')
data_path = "../data/"
post_path = "../data/"
from utility import uniquify, reg_log
from feat_extractor import feature_extract, post_extract 
from architecture import class_model, class_model_cnn

In [3]:
# naming conventions of the data files
# geometry and priors
malp_min, malp_max = 0.1, 4.5
tmalp_min, tmalp_max = 0.05, 500 # SHiP
z_min, z_max, z_cal, l_x, l_y= 32, 82, 93, 2, 3 # SHiP
x_min, x_max = -l_x, l_x
y_min, y_max = -l_y, l_y

par_lab = "m_"+str(malp_min)+"_"+str(malp_max)+"_tm_"+str(tmalp_min)+"_"+str(tmalp_max)+"_"
geo_lab = "c_"+str(z_min)+"_"+str(z_max)+"_"+str(l_x)+"_"+str(l_y)


In [4]:
def prior_m():
    return sts.uniform(loc=np.log10(malp_min), scale= np.log10(malp_max/malp_min))

def prior_tm():
    return sts.uniform(loc=np.log10(tmalp_min), scale= np.log10(tmalp_max/tmalp_min))

## Training of ECO hunt

In [6]:
smear = 1 # 0 or 1 for small or large smearing

if smear ==0:
    sigs, labsm = [0.001,0.01,0.005, 0.005 ], "small" # small uncertainty case
elif smear == 1:
    sigs, labsm = [0.001,0.05,0.01,  0.01  ], "large" # large uncertainty case

In [9]:
dirname = uniquify("../models/model_01_"+labsm+"_")
os.mkdir(dirname)

In [10]:
nobs =2

trainfile0 = data_path + "event_train_00_"+par_lab+geo_lab+".csv"
trainfile1 = data_path + "event_train_11_"+par_lab+geo_lab+".csv"

# we do not save the masses and lifetimes for training (but they can be used for debugging)

# extract features from bkg and sig
feats=feature_extract(trainfile0, sigs[0], sigs[1], sigs[2], sigs[3], Eres=1,  x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max)
features = np.vstack(([feats.extract_llo(iOBS) for iOBS in range(nobs)]))  
x0 = features.T

feats=feature_extract(trainfile1, sigs[0], sigs[1], sigs[2], sigs[3], Eres=1,  x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max)
features = np.vstack(([feats.extract_llo(iOBS) for iOBS in range(nobs)]))  
x1 = features.T


X = np.vstack((x0,x1))
y=np.hstack((np.zeros(len(x0)),np.ones(len(x1))))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

xzscaler=StandardScaler().fit(X_train) 
X_train=xzscaler.transform(X_train)
X_val=xzscaler.transform(X_val)

del X, x0, x1, feats, features # just in case, to save some memory

with open(dirname+'/xzscaler.pkl', 'wb') as file_t:
    pickle.dump(xzscaler, file_t)


In [11]:
# parameters have been optimized with kerastuner
decay_steps = 500
decay_rate = 0.95
batch_size = 4096
min_d = 0.005
patience = 20

bn, n_lay, n_units, lr = 1, 4, 64, 2.4e-2


meta_par = {"n_units": n_units, "n_lay": n_lay, "lr": lr, "decay_steps": decay_steps, "decay_rate": decay_rate, "bn": bn, "batch_size": batch_size, "sigs": sigs}

with open(dirname+'/meta_par.pkl', 'wb') as file_t:
    pickle.dump(meta_par, file_t)


# call existing model-building code with the hyperparameter values.
model = class_model(
    n_units=n_units, n_lay=n_lay,  lr=lr, decay_steps=decay_steps, decay_rate=decay_rate, bn=bn
)


early= tf.keras.callbacks.EarlyStopping(
monitor="val_accuracy",
min_delta=min_d,
patience=patience,
restore_best_weights=True,
)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=batch_size, callbacks=[early, keras.callbacks.TensorBoard(dirname+"/logs")])

Epoch 1/100
98/98 [==============================] - 3s 12ms/step - loss: 0.5813 - accuracy: 0.6714 - val_loss: 1.1628 - val_accuracy: 0.5703
Epoch 2/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4673 - accuracy: 0.7683 - val_loss: 1.3113 - val_accuracy: 0.5841
Epoch 3/100
98/98 [==============================] - 1s 9ms/step - loss: 0.4333 - accuracy: 0.7929 - val_loss: 0.8568 - val_accuracy: 0.6658
Epoch 4/100
98/98 [==============================] - 1s 9ms/step - loss: 0.4204 - accuracy: 0.8010 - val_loss: 0.6861 - val_accuracy: 0.7041
Epoch 5/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4118 - accuracy: 0.8071 - val_loss: 0.5827 - val_accuracy: 0.7267
Epoch 6/100
98/98 [==============================] - 1s 11ms/step - loss: 0.4041 - accuracy: 0.8121 - val_loss: 0.5177 - val_accuracy: 0.7543
Epoch 7/100
98/98 [==============================] - 1s 10ms/step - loss: 0.4012 - accuracy: 0.8140 - val_loss: 0.4779 - val_accuracy: 0.7729
Epoch 8/

In [12]:
# check of the performance of the model before saving (this is the model which will be saved)
model_index = np.argmax(history.history["val_accuracy"])
print(history.history["accuracy"][model_index], history.history["val_accuracy"][model_index])

0.8579300045967102 0.8519399762153625


In [25]:
# uncomment to save
# two versions for improved tensorflow compatibility
# model.save(dirname+"/model.keras")
# model.save(dirname+"/model.tf")

INFO:tensorflow:Assets written to: models/ship/model_01_large_4/model.tf/assets


INFO:tensorflow:Assets written to: models/ship/model_01_large_4/model.tf/assets


## train posterior extractor

In [13]:
smear = 0 # small or large uncertainty

if smear == 0:
    sigs, labsm = [0.001,0.01,0.005, 0.005 ], "small" # small uncertainty case
elif smear == 1:
    sigs, labsm = [0.001,0.05,0.01,  0.01  ], "large" # large uncertainty case

In [15]:
dirname = uniquify("../models/model_post_"+labsm+"_")
os.mkdir(dirname)

In [16]:
# here we need the mass to get the posterior
trainfile = data_path+"event_train_post_"+par_lab+geo_lab+".csv"

feats=feature_extract(trainfile, sigs[0], sigs[1], sigs[2], sigs[3], Eres=1,  x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max)
features = feats.extract_llo(0)
x = features.T

malp, _ = feats.extract_model(0)

# create wrong mass and event combination for training
Nsam = int(len(malp)/2)
malp_true = np.log10(malp[:Nsam])
malp_fake = prior_m().rvs(Nsam)
z = np.hstack((malp_true, malp_fake)).reshape(-1,1)

y=np.hstack((np.ones(Nsam),np.zeros(Nsam)))

X=np.hstack((z, x))
y=np.hstack((np.ones(Nsam),np.zeros(Nsam)))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

del X, z, feats, features # just in case

xzscaler=StandardScaler().fit(X_train) 
X_train=xzscaler.transform(X_train)
X_val=xzscaler.transform(X_val)


with open(dirname+'/xzscaler.pkl', 'wb') as file_t:
    pickle.dump(xzscaler, file_t)


In [17]:
# parameters have been optimized with kerastuner


decay_steps = 500
decay_rate = 0.95
bn = 0
batch_size = 4096
min_d = 0.005
patience = 20

n_lay, n_units, lr = 4, 64, 2.4e-2

meta_par = {"n_units": n_units, "n_lay": n_lay, "lr": lr, "decay_steps": decay_steps, "decay_rate": decay_rate, "bn": bn, "batch_size": batch_size, "sigs": sigs}

with open(dirname+'/meta_par.pkl', 'wb') as file_t:
    pickle.dump(meta_par, file_t)


# call existing model-building code with the hyperparameter values.
model = class_model(
    n_units=n_units, n_lay=n_lay,  lr=lr, decay_steps=decay_steps, decay_rate=decay_rate, bn=bn
)



early= tf.keras.callbacks.EarlyStopping(
monitor="val_loss",
min_delta=min_d,
patience=patience,
restore_best_weights=True,
)


history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=batch_size, callbacks=[early, keras.callbacks.TensorBoard(dirname+"/logs")])


Epoch 1/2
49/49 [==============================] - 1s 9ms/step - loss: 0.5247 - accuracy: 0.7151 - val_loss: 0.4014 - val_accuracy: 0.8099
Epoch 2/2
49/49 [==============================] - 0s 7ms/step - loss: 0.3689 - accuracy: 0.8334 - val_loss: 0.3474 - val_accuracy: 0.8460


In [18]:
# check of the performance of the model before saving (this is the model which will be saved)
model_index = np.argmin(history.history["val_loss"])
print(history.history["loss"][model_index], history.history["val_loss"][model_index])

0.3688754439353943 0.3474162220954895


In [ ]:
# uncomment to save
# two versions for improved tensorflow compatibility
#model.save(dirname+"/model.keras")
#model.save(dirname+"/model.tf")

## train EPO hunt

In [19]:
smear = 0

if smear == 0:
    sigs, labsm = [0.001,0.01,0.005, 0.005 ], "small" # small uncertainty case
elif smear == 1:
    sigs, labsm = [0.001,0.05,0.01,  0.01  ], "large" # large uncertainty case

In [20]:
n_units = 64
n_lay = 4
lr = 0.5e-3
decay_steps = 500
decay_rate = 0.95
bn = 0
maxp = 1
filt_arr = [64, 64, 128]
batch_size = 4096
pref = -3

meta_par = {"maxp": maxp, "pref": pref, "n_units": n_units, "n_lay": n_lay, "lr": lr, "decay_steps": decay_steps, "decay_rate": decay_rate, "bn": bn, "batch_size": batch_size, "sigs": sigs, "filters": filt_arr}


In [6]:
# these files are not included as they are too large
# they can be obtained with post_extract
feats = post_extract(data_path + "post_train_0_"+labsm+".csv")
x0= feats.extract_post(2)
del feats


feats = post_extract(data_path + "post_train_1_"+labsm+".csv")
x1= feats.extract_post(2)
del feats

x0 = x0.reshape(250000,2,200)
x1 = x1.reshape(250000,2,200)

X = reg_log(np.vstack((x0,x1)), meta_par["pref"])
y=np.hstack((np.zeros(len(x0)),np.ones(len(x1))))

del x0, x1
# no need to save maxs and mins as the posterior are separately normalized

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
del X, y

In [19]:
dirname = uniquify("../models/model_post_01_"+labsm+"_")
os.mkdir(dirname)

In [20]:
with open(dirname+'/meta_par.pkl', 'wb') as file_t:
    pickle.dump(meta_par, file_t)


# call existing model-building code with the hyperparameter values.
model = class_model_cnn(
    n_units=n_units, n_lay=n_lay,  lr=lr, decay_steps=decay_steps, decay_rate=decay_rate, bn=bn, filters=filt_arr, maxp = maxp
)

early= tf.keras.callbacks.EarlyStopping(
monitor="val_accuracy",
min_delta=0.005,
patience=20,
restore_best_weights=True,
)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=batch_size, callbacks=[early, keras.callbacks.TensorBoard(dirname+"/logs")])

Epoch 1/100
98/98 [==============================] - 7s 54ms/step - loss: 0.4464 - accuracy: 0.7857 - val_loss: 0.3353 - val_accuracy: 0.8658
Epoch 2/100
98/98 [==============================] - 5s 49ms/step - loss: 0.3204 - accuracy: 0.8705 - val_loss: 0.3137 - val_accuracy: 0.8719
Epoch 3/100
98/98 [==============================] - 5s 50ms/step - loss: 0.3116 - accuracy: 0.8724 - val_loss: 0.3093 - val_accuracy: 0.8730
Epoch 4/100
98/98 [==============================] - 5s 50ms/step - loss: 0.3079 - accuracy: 0.8729 - val_loss: 0.3068 - val_accuracy: 0.8732
Epoch 5/100
98/98 [==============================] - 5s 50ms/step - loss: 0.3065 - accuracy: 0.8737 - val_loss: 0.3093 - val_accuracy: 0.8716
Epoch 6/100
98/98 [==============================] - 5s 52ms/step - loss: 0.3061 - accuracy: 0.8735 - val_loss: 0.3066 - val_accuracy: 0.8732
Epoch 7/100
98/98 [==============================] - 5s 50ms/step - loss: 0.3041 - accuracy: 0.8743 - val_loss: 0.3048 - val_accuracy: 0.8740
Epoch 

In [21]:
# check of the performance of the model before saving (this is the model which will be saved)
model_index = np.argmax(history.history["val_accuracy"])
print(history.history["accuracy"][model_index], history.history["val_accuracy"][model_index])

0.8756725192070007 0.8745099902153015


In [22]:
# model.save(dirname+"/model.keras")
# model.save(dirname+"/model.tf")

INFO:tensorflow:Assets written to: models/ship/model_post_01_large_4/model.tf/assets


INFO:tensorflow:Assets written to: models/ship/model_post_01_large_4/model.tf/assets
